In [1]:
import requests
import pandas as pd
import time
pd.set_option("display.max_columns", None)
import datetime
from datetime import timezone, timedelta
from geopy.distance import geodesic #計算GPS兩端點距離
import folium #GPS數據視覺化
from global_land_mask import globe #確認GPS位置是否在陸地
import warnings
warnings.filterwarnings('ignore') # 關閉運行的警告
pd.set_option("mode.chained_assignment", None)
import pprint
import json
import tkinter as tk
from tkinter import filedialog
import gc
import sqlite3
import httpx
import numpy as np
import os

In [49]:
dtime = datetime.datetime(2023, 4, 28, 11, 59)
while True:
    cur_time = datetime.datetime.now()
    if dtime <= cur_time:
        print("Now is {}".format(cur_time))
        break
    time.sleep(1)

Now is 2023-04-28 11:59:00.978042


In [7]:
start_date = datetime.date(2021, 7, 1)

In [17]:
datetime.datetime.strptime(start_date.strftime("%Y-%m-%d"), "%Y-%m-%d")

datetime.datetime(2021, 7, 1, 0, 0)

In [8]:
start_date + timedelta(days=3)

datetime.date(2021, 7, 4)

## 每次執行前先更新路線和業者資訊

In [2]:
icb_route_api = "https://linkmotc.map8.zone/app/data?modelName=route_coach&skip=0&limit=&whereOptions="
icb_operator_api = "https://linkmotc.map8.zone/app/data?modelName=operator_coach&skip=0&limit=&whereOptions="

In [3]:
def update_api(api):
    res = httpx.get(api)
    df = pd.read_json(res.text)
    return df
    

## 從TDX歷史資料匯入資料庫
### server: TDX.db
### table: ICB

In [76]:
fetch_date = pd.DataFrame(pd.date_range(start="2021-07-01", end=datetime.date.today() - timedelta(days=1), freq="D"), columns=["start_date"])

In [81]:
fetch_date["end_date"] = fetch_date["start_date"] + timedelta(days=7)

In [103]:
tdx = TDX()
base_url = "https://tdx.transportdata.tw/api/historical"
endpoint = "/v2/Historical/Bus/RealTimeByFrequency/InterCity"
query = "?Dates=2021-07-01~2021-07-07&%24format=JSONL"
url = f"{base_url}{endpoint}{query}"
operators = tdx.get_response(url)

In [ ]:
operators

'https://tdx.transportdata.tw/api/historical/v2/Historical/Bus/RealTimeByFrequency/InterCity?Dates=2021-07-01~2021-07-08&%24format=JSONL'

In [87]:
zip(fetch_date["start_date"], fetch_date["end_date"])

## 20230410 Test the historical csv file extract
### For monitoring daily vehicle mileage, just need to keep the columns of:
- *PlateNumb*
- *OperatorID*
- *PositionLon*
- *PositionLat*
- *GeoHash*
- *Speed*
- *UpdateTime*

### Finally only "PlateNumb", "OperatorID", "Distance", "Date" and "AVG. Speed" will be written to database


In [117]:
# 測試讀取檔案路徑list存進txt，看看中斷之後再次讀取是否OK
root = tk.Tk()
root.withdraw()
paths = filedialog.askopenfilenames()

# 將開啟的路徑都寫進去tdx_path.txt
with open("tdx_paths.txt", "w") as f:
    for path in paths:
        f.write(path + "\n")

In [118]:
# 打開tdx_paths.txt並讀取所有路徑
with open("tdx_paths.txt", "r") as f:
    lines = f.readlines()

#當路徑list的長度大於0，執行動作，並將路徑從list中刪除，一旦中斷執行，就將剩下還未執行到的路徑list重新寫回檔案中待下次繼續執行
while len(lines) > 0:
    path = lines[0].rstrip("\n")

    try:
        df = pd.read_excel(path)
        print("讀取完畢")
        lines.pop(0)
        
    except:
        with open("tdx_paths.txt", "w") as f:
            for line in lines:
                f.write(line)
        break

# os.remove("tdx_paths.txt")

讀取完畢


In [7]:
df = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\TDX下載\ICB\unzip files\公路客運定時歷史資料(A1)2023-03-31.CSV")

In [9]:
keep_columns = ["PlateNumb", "OperatorID", "RouteUID", "Direction", "PositionLon", "PositionLat", "UpdateTime"]
data = df[keep_columns]
data

,PlateNumb,OperatorID,RouteUID,Direction,PositionLon,PositionLat,UpdateTime
0,947-FS,21,THB0701,返程,120.387420,23.697410,2023-03-31 09:20:09+08:00
1,943-FS,21,THB0701,去程,120.200305,23.698688,2023-03-31 09:20:14+08:00
2,947-FS,21,THB0701,返程,120.387417,23.697197,2023-03-31 09:20:29+08:00
3,943-FS,21,THB0701,去程,120.200305,23.698688,2023-03-31 09:20:34+08:00
4,947-FS,21,THB0701,返程,120.387410,23.697168,2023-03-31 09:20:49+08:00
...,...,...,...,...,...,...,...
4116266,931-V2,28,THB9189,返程,120.799827,21.945063,2023-03-31 18:04:59+08:00
4116267,FAB-950,45,THB9189,去程,120.333023,22.656860,2023-03-31 18:04:59+08:00
4116268,FAB-955,45,THB9189,返程,120.642723,22.304065,2023-03-31 18:04:59+08:00
4116269,912-U3,45,THB9189,去程,120.808573,21.938093,2023-03-31 18:05:04+08:00


In [10]:
data["Coordinates"] = data[["PositionLat", "PositionLon"]].apply(tuple, axis=1)

In [11]:
data["Land"] = globe.is_land(data["PositionLat"], data["PositionLon"])
data = data[data["Land"] == True]
data["NextCoordinates"] = data.groupby("PlateNumb")["Coordinates"].shift(-1)
data["Distance"] = data.apply(lambda data: geodesic(data["Coordinates"], data["NextCoordinates"]).meters if pd.notna(data["NextCoordinates"]) else 0, axis=1) / 1000
data = data[data["Distance"] <= 2.5]
data["Time"] = pd.to_datetime(data["UpdateTime"].map(lambda x: x.split("+")[0]), format="%Y-%m-%dT%H:%M")

result = data.groupby(["PlateNumb", "OperatorID"]).agg({"Distance": "sum", "Time": lambda x: (x.max() - x.min()) / pd.Timedelta(hours=1)}).reset_index()
result["Date"] = data["Time"].max().strftime("%Y-%m-%d")
result["AVG. Speed"] = result["Distance"] / result["Time"]
result["AVG. Speed"].fillna(0, inplace=True)
result[["Distance", "Time", "AVG. Speed"]] = result[["Distance", "Time", "AVG. Speed"]].applymap(lambda x: round(x, 1))

In [12]:
# result["路線"] = result["RouteUID"].map(route_dict)
result["業者名稱"] = result["OperatorID"].map(operator_dict)

In [13]:
result["業者中文名稱"] = result["業者名稱"].map(lambda x: x[0])
result["業者英文名稱"] = result["業者名稱"].map(lambda x: x[-1])

In [14]:
result

,PlateNumb,OperatorID,Distance,Time,Date,AVG. Speed,業者名稱,業者中文名稱,業者英文名稱
0,001-U6,45,186.4,10.6,2023-03-31,17.5,"(國光客運, Kuo-Kuang Bus Co., Ltd.)",國光客運,"Kuo-Kuang Bus Co., Ltd."
1,002-U6,45,245.7,11.3,2023-03-31,21.8,"(國光客運, Kuo-Kuang Bus Co., Ltd.)",國光客運,"Kuo-Kuang Bus Co., Ltd."
2,003-U6,45,0.0,8.4,2023-03-31,0.0,"(國光客運, Kuo-Kuang Bus Co., Ltd.)",國光客運,"Kuo-Kuang Bus Co., Ltd."
3,007-U6,45,492.1,15.2,2023-03-31,32.3,"(國光客運, Kuo-Kuang Bus Co., Ltd.)",國光客運,"Kuo-Kuang Bus Co., Ltd."
4,008-U6,45,259.7,7.0,2023-03-31,37.0,"(國光客運, Kuo-Kuang Bus Co., Ltd.)",國光客運,"Kuo-Kuang Bus Co., Ltd."
...,...,...,...,...,...,...,...,...,...
2875,KKB-8731,29,61.7,1.4,2023-03-31,45.5,"(屏東客運, Pingtung Bus Co., Ltd.)",屏東客運,"Pingtung Bus Co., Ltd."
2876,KKB-8735,29,267.2,12.9,2023-03-31,20.7,"(屏東客運, Pingtung Bus Co., Ltd.)",屏東客運,"Pingtung Bus Co., Ltd."
2877,KKB-8736,29,218.0,12.0,2023-03-31,18.2,"(屏東客運, Pingtung Bus Co., Ltd.)",屏東客運,"Pingtung Bus Co., Ltd."
2878,KKB-8750,29,209.9,13.0,2023-03-31,16.1,"(屏東客運, Pingtung Bus Co., Ltd.)",屏東客運,"Pingtung Bus Co., Ltd."


In [15]:
four = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\市場資訊\大車監理數據.xlsx", sheet_name="Sheet1")

In [16]:
four["廠牌"] = four["廠牌"].fillna("")
four["車體"] = four["車體"].fillna("")

In [17]:
brand_dict = dict(zip(four["車號"], four["廠牌"]))
body_dict = dict(zip(four["車號"], four["車體"]))

In [18]:
result["廠牌"] = result["PlateNumb"].map(brand_dict)
result["車體"] = result["PlateNumb"].map(body_dict)

In [19]:
result = result[["PlateNumb", "業者中文名稱", "業者英文名稱", "廠牌", "車體", "Distance", "Time", "Date", "AVG. Speed"]]

In [20]:
result[result["廠牌"].isna()]

,PlateNumb,業者中文名稱,業者英文名稱,廠牌,車體,Distance,Time,Date,AVG. Speed
74,069-VV,首都客運,"Capital Bus Co., Ltd.",NaN,NaN,318.1,16.3,2023-03-31,19.6
86,075-VV,首都客運,"Capital Bus Co., Ltd.",NaN,NaN,202.5,14.6,2023-03-31,13.8
89,076-VV,首都客運,"Capital Bus Co., Ltd.",NaN,NaN,320.7,14.0,2023-03-31,23.0
100,082-V5,三重客運,"San Chung Bus Co., Ltd.",NaN,NaN,206.2,14.9,2023-03-31,13.8
102,085-FN,新竹客運,"Hsinchu Bus Co., Ltd.",NaN,NaN,287.6,11.2,2023-03-31,25.6
...,...,...,...,...,...,...,...,...,...
1928,FAF-503,和欣客運,"Ho-Hsin Bus Co., Ltd.",NaN,NaN,212.5,5.9,2023-03-31,36.0
1929,FAF-507,和欣客運,"Ho-Hsin Bus Co., Ltd.",NaN,NaN,191.3,4.0,2023-03-31,48.0
1930,FAF-802,臺西客運,"Taisi Bus Co., Ltd.",NaN,NaN,244.4,14.6,2023-03-31,16.8
1931,FAF-805,臺西客運,"Taisi Bus Co., Ltd.",NaN,NaN,322.9,11.1,2023-03-31,29.1


In [22]:
df

,PlateNumb,OperatorID,OperatorNo,RouteUID,RouteID,RouteNameZh_tw,RouteNameEn,SubRouteUID,SubRouteID,SubRouteNameZh_tw,SubRouteNameEn,Direction,PositionLon,PositionLat,GeoHash,Speed,Azimuth,DutyStatus,BusStatus,MessageType,GPSTime,TransTime,SrcRecTime,SrcTransTime,SrcUpdateTime,UpdateTime
0,947-FS,21,1408,THB0701,701,701,701,THB070102,070102,0701,0701,返程,120.387420,23.697410,wsjwg3zrq,21,186,開始,正常,定期,2023-03-31 09:20:06+08:00,NaN,2023-03-31 09:20:06+08:00,2023-03-31 09:20:06+08:00,NaN,2023-03-31 09:20:09+08:00
1,943-FS,21,1408,THB0701,701,701,701,THB070101,070101,0701,0701,去程,120.200305,23.698688,wsjqz4pny,0,267,開始,正常,定期,2023-03-31 09:20:12+08:00,NaN,2023-03-31 09:20:13+08:00,2023-03-31 09:20:13+08:00,NaN,2023-03-31 09:20:14+08:00
2,947-FS,21,1408,THB0701,701,701,701,THB070102,070102,0701,0701,返程,120.387417,23.697197,wsjwg3zqn,0,186,開始,正常,定期,2023-03-31 09:20:26+08:00,NaN,2023-03-31 09:20:26+08:00,2023-03-31 09:20:26+08:00,NaN,2023-03-31 09:20:29+08:00
3,943-FS,21,1408,THB0701,701,701,701,THB070101,070101,0701,0701,去程,120.200305,23.698688,wsjqz4pny,0,267,開始,正常,定期,2023-03-31 09:20:32+08:00,NaN,2023-03-31 09:20:33+08:00,2023-03-31 09:20:33+08:00,NaN,2023-03-31 09:20:34+08:00
4,947-FS,21,1408,THB0701,701,701,701,THB070102,070102,0701,0701,返程,120.387410,23.697168,wsjwg3zqj,15,186,開始,正常,定期,2023-03-31 09:20:46+08:00,NaN,2023-03-31 09:20:46+08:00,2023-03-31 09:20:47+08:00,NaN,2023-03-31 09:20:49+08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116266,931-V2,28,1003,THB9189,9189,9189,9189,THB918902,918902,9189,9189,返程,120.799827,21.945063,wevuufgxr,0,0,正常,正常,定期,2023-03-31 18:04:57+08:00,NaN,2023-03-31 18:04:58+08:00,2023-03-31 18:04:58+08:00,NaN,2023-03-31 18:04:59+08:00
4116267,FAB-950,45,1101,THB9189,9189,9189,9189,THB918901,918901,9189,9189,去程,120.333023,22.656860,wsj8fhxcf,36,155,正常,客滿,定期,2023-03-31 18:04:54+08:00,NaN,2023-03-31 18:04:55+08:00,2023-03-31 18:04:55+08:00,NaN,2023-03-31 18:04:59+08:00
4116268,FAB-955,45,1101,THB9189,9189,9189,9189,THB918902,918902,9189,9189,返程,120.642723,22.304065,wevyck36e,7,338,開始,正常,定期,2023-03-31 18:04:54+08:00,NaN,2023-03-31 18:04:55+08:00,2023-03-31 18:04:55+08:00,NaN,2023-03-31 18:04:59+08:00
4116269,912-U3,45,1101,THB9189,9189,9189,9189,THB918901,918901,9189,9189,去程,120.808573,21.938093,wevuv1dne,59,333,開始,正常,定期,2023-03-31 18:04:59+08:00,NaN,2023-03-31 18:05:00+08:00,2023-03-31 18:05:00+08:00,NaN,2023-03-31 18:05:04+08:00


In [29]:
data["Land"] = globe.is_land(data["PositionLat"], data["PositionLon"])
data = data[data["Land"] == True]
data["NextCoordinates"] = data.groupby("PlateNumb")["Coordinates"].shift(-1)
data["Distance"] = data.apply(lambda data: geodesic(data["Coordinates"], data["NextCoordinates"]).meters if pd.notna(data["NextCoordinates"]) else 0, axis=1) / 1000
data = data[data["Distance"] <= 2.5]
data["Time"] = pd.to_datetime(data["UpdateTime"].map(lambda x: x.split("+")[0]), format="%Y-%m-%dT%H:%M")

result = data.groupby(["PlateNumb", "OperatorID"]).agg({"Distance": "sum", "Time": lambda x: (x.max() - x.min()) / pd.Timedelta(hours=1)}).reset_index()
result["Date"] = data["Time"].max().strftime("%Y-%m-%d")
result["AVG. Speed"] = result["Distance"] / result["Time"]
result["AVG. Speed"].fillna(0, inplace=True)
result[["Distance", "Time", "AVG. Speed"]] = result[["Distance", "Time", "AVG. Speed"]].applymap(lambda x: round(x, 1))

In [51]:
# 第一次初始化table才需要執行
con = sqlite3.connect("./tdx.db")
cur = con.cursor()

cur.execute("create table ICB (PlateNumb TEXT, OperatorID TEXT, Distance REAL, Time REAL, Date TEXT, AVG_Speed REAL)")
con.commit()
con.close()

In [52]:
con = sqlite3.connect("./tdx.db")
cur = con.cursor()
for index, row in result.iterrows():
    # print(tuple(row))
    cur.execute('INSERT INTO ICB (PlateNumb, OperatorID, Distance, Time, Date, AVG_Speed) values (?, ?, ?, ?, ?, ?)', tuple(row))
con.commit()
con.close()
    

In [8]:
root = tk.Tk()
root.withdraw()
paths = filedialog.askopenfilenames()

In [12]:
con = sqlite3.connect("./tdx.db")
cur = con.cursor()

keep_columns = ["PlateNumb", "OperatorCN_Name", "OperatorEN_Name", "Direction", "DepartureStop", "DestinationStop", "PositionLon", "PositionLat", "GeoHash", "Speed", "Azimuth", "DutyStatus", "BusStatus", "MessageType", "UpdateTime"]

for path in paths:
    print(path)
    df = pd.read_csv(path)
    df["OperatorName"] = df["OperatorID"].map(oprator_dict)
    df[["OperatorCN_Name", "OperatorEN_Name"]] = pd.DataFrame(df["OperatorName"].tolist(), index=df.index)
    df["RouteName"] = df["RouteUID"].map(route_dict)
    df[["DepartureStop", "DestinationStop"]] = pd.DataFrame(df["RouteName"].tolist(), index=df.index)
    df = df[keep_columns]
    for index, row in df.iterrows():
        cur.execute('INSERT INTO ICB (PlateNumb, OperatorCN_Name, OperatorEN_Name, Direction, DepartureStop, DestinationStop, PositionLon, PositionLat, GeoHash, Speed, Azimuth, DutyStatus, BusStatus, MessageType, UpdateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
    del df
    gc.collect()
    

con.commit()
con.close()
    

D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/公路客運定時歷史資料(A1)[CSV]2023-01-01_2023-03-07/公路客運定時歷史資料(A1)2023-01-01.CSV
D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/公路客運定時歷史資料(A1)[CSV]2023-01-01_2023-03-07/公路客運定時歷史資料(A1)2023-01-02.CSV
D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/公路客運定時歷史資料(A1)[CSV]2023-01-01_2023-03-07/公路客運定時歷史資料(A1)2023-01-03.CSV
D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/公路客運定時歷史資料(A1)[CSV]2023-01-01_2023-03-07/公路客運定時歷史資料(A1)2023-01-04.CSV
D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/公路客運定時歷史資料(A1)[CSV]2023-01-01_2023-03-07/公路客運定時歷史資料(A1)2023-01-05.CSV
D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/公路客運定時歷史資料(A1)[CSV]2023-01-01_2023-03-07/公路客運定時歷史資料(A1)2023-01-06.CSV
D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/公路客運定時歷史資料(A1)[CSV]2023-01-01_2023-03-07/公路客運定時歷史資料(A1)2023-01-07.CSV
D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/公路客運定時歷史資料(A1)[CSV]2023-01-01_2023-03-07/公路客運定時歷史資料(A1)2023-01-08.CSV
D:/kc.hsu/OneDrive - Bridgestone/數據/TDX/TDX下載/公路客運定時歷史資料(A1)[CSV]2023-01

KeyboardInterrupt: 

## 從api匯入資料

In [2]:
class TDX():
    
    '''
    將常用的屬性放在類別屬性中方便後續呼叫.
    '''
    client_id : str = "kc.hsu-e51d77a0-5d9b-418b"
    client_secret = "afbb6a0a-f69d-4556-bd0b-4e53eae46379"
    cities = ["Taipei", "NewTaipei", "Taoyuan", "Taichung", "Tainan", "Kaohsiung", "Keelung", "Hsinchu", "HsinchuCounty", "MiaoliCounty", "ChanghuaCounty", "NantouCounty", "YunlinCounty", "ChiayiCounty", "ChiayiCounty", "Chiayi", "PingtungCounty", "YilanCounty", "HualienCounty", "TaitungCounty", "KinmenCounty", "PenghuCounty", "LienchiangCounty"]

    
    def __init__(self, client_id=client_id, client_secret=client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
        self.token = self.get_token()
        
    def get_token(self):
        token_url = "https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token"
        headers = {'content-type': 'application/x-www-form-urlencoded'}
        data = {
            'grant_type': 'client_credentials',
            'client_id': self.client_id,
            'client_secret': self.client_secret
        }
        response = requests.post(token_url, headers=headers, data=data)
        return response.json()['access_token']
    
    def get_response(self, url):
        headers = {"authorization": f"Bearer {self.token}"}
        response = requests.get(url, headers=headers)
        return response.json()
        # return json.loads(response.text)
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        

### 市區公車業者名單抓取

In [11]:
dfs = []
cities = ["Taipei", "NewTaipei", "Taoyuan", "Taichung", "Tainan", "Kaohsiung", "Keelung", "Hsinchu", "HsinchuCounty", "MiaoliCounty", "ChanghuaCounty", "NantouCounty", "YunlinCounty", "ChiayiCounty", "ChiayiCounty", "Chiayi", "PingtungCounty", "YilanCounty", "HualienCounty", "TaitungCounty", "KinmenCounty", "PenghuCounty", "LienchiangCounty"]
for city in cities:
    tdx = TDX()
    base_url = "https://tdx.transportdata.tw/api/basic"
    endpoint = "/v2/Bus/Operator/City/"
    url = f"{base_url}{endpoint}{city}"
    operators = tdx.get_response(url)
    dfs.append(pd.DataFrame.from_records(operators))
df = pd.concat(dfs)
df["OperatorNameZh_tw"] = df["OperatorName"].map(lambda x: x["Zh_tw"])
df.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\市區公車業者名單.xlsx", index=False)

### 國道客運業者名單抓取

In [10]:
tdx = TDX()
base_url = "https://tdx.transportdata.tw/api/basic"
endpoint = "/v2/Bus/Operator/InterCity"
url = f"{base_url}{endpoint}"
operators = tdx.get_response(url)
intercity_operators = pd.DataFrame.from_records(operators)
intercity_operators["OperatorNameZh_tw"] = intercity_operators["OperatorName"].map(lambda x: x["Zh_tw"])
intercity_operators.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\國道客運業者名單.xlsx", index=False)

## 國道客運即時數據抓取

In [7]:
tdx = TDX()
base_url = "https://tdx.transportdata.tw/api/basic"
endpoint = "/v2/Bus/RealTimeByFrequency/InterCity"
url = base_url + endpoint
result = pd.DataFrame.from_records(tdx.get_response(url))
result = result[['PlateNumb', 'OperatorID', 'RouteUID', 'RouteID',
       'SubRouteUID', 'SubRouteID', 'Direction', 'BusPosition',
       'Speed', 'Azimuth', 'DutyStatus', 'BusStatus', 'MessageType', 'GPSTime',
       'SrcRecTime', 'SrcTransTime', 'UpdateTime']]

In [9]:
result["Longitude"] = result["BusPosition"].map(lambda x: x["PositionLon"])
result["Latitude"] = result["BusPosition"].map(lambda x: x["PositionLat"])

In [11]:
result = result[['PlateNumb', 'OperatorID', 'RouteUID', 'RouteID',
       'SubRouteUID', 'SubRouteID', 'Direction', 'Longitude', 'Latitude',
       'Speed', 'Azimuth', 'DutyStatus', 'BusStatus', 'MessageType', 'GPSTime',
       'SrcRecTime', 'SrcTransTime', 'UpdateTime']]

In [16]:
result["Coordinates"] = result[["Longitude", "Latitude"]].apply(tuple, axis=1)

In [21]:
result.groupby(["OperatorID"])["Coordinates"].shift(-1)

0       (120.786511666667, 24.4588066666667)
1       (121.537478333333, 24.9586483333333)
2       (121.438336666667, 25.0859366666667)
3               (120.298941666667, 23.56911)
4             (120.326695, 22.6297216666667)
                        ...                 
1816                                     NaN
1817            (121.696346666667, 25.15598)
1818                                     NaN
1819    (121.756978333333, 25.1282183333333)
1820                                     NaN
Name: Coordinates, Length: 1821, dtype: object

In [14]:
result.to_excel(r"C:\Users\kc.hsu\Desktop\國道客運API範例內容.xlsx", index=False)

In [ ]:
def time_range_data(st_year=0, st_month=0, st_day=0, st_hour=0, st_min=0, end_year=0, end_month=0, end_day=0, end_hour=0, end_min=0):
    st_time = datetime.datetime(st_year, st_month, st_day, st_hour, st_min).replace(tzinfo=datetime.timezone.utc).astimezone(timezone(timedelta(hours=0))).replace(microsecond=0).isoformat()
    end_time = datetime.datetime(end_year, end_month, end_day, end_hour, end_min).replace(tzinfo=datetime.timezone.utc).astimezone(timezone(timedelta(hours=0))).replace(microsecond=0).isoformat()
    dfs = []
    tdx = TDX()
    while True:
        current_time = datetime.datetime.utcnow().replace(tzinfo=datetime.timezone.utc).astimezone(tz=None).replace(microsecond=0).isoformat()
        if st_time <= current_time <= end_time:
            base_url = "https://tdx.transportdata.tw/api/basic"
            endpoint = "/v2/Bus/RealTimeByFrequency/InterCity" # batch update(Frequency: 60sec.)
            # filter = "Direction eq 1"  # 順逆行: [0:'順行', 1:'逆行']
            url = f"{base_url}{endpoint}"
            try:
                result = pd.DataFrame.from_records(tdx.get_response(url))
            except:
                tdx.token = tdx.get_token()
                continue
            else:
                result["Longitude"] = result["BusPosition"].map(lambda x: x["PositionLon"])
                result["Latitude"] = result["BusPosition"].map(lambda x: x["PositionLat"])
                dfs.append(result)
                time.sleep(60) # Update the data every 60 sec.
        elif current_time < st_time:
            # print("等待開始")
            continue
        else:
            print("抓取結束")
            result = pd.concat(dfs)
            # print(result)
            break
    # result = result[["PlateNumb", "OperatorID", "RouteUID", "Direction", "DutyStatus", "BusStatus", "MessageType", "Latitude", "Longitude", "UpdateTime"]]
    return result
    

In [ ]:
def icb_summary(df):
    direction_dict = {0 : "去程(南下)", 1 : "返程(北上)", 2 : "迴圈(雙向)"}
    busStatus_dict = {0: "正常", 1 : "車禍", 2 : "故障", 3 : "塞車", 4 : "緊急救援", 5 : "加油", 98 : "偏移路線", 99 : "非營業狀態", 100 : "客滿", 101 : "包車出租", 255 : "未知"}
    busDutyStatus_dict = {0: "正常", 1 : "開始", 2 : "結束"}
    operator = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\20221026_國道客運業者名單.xlsx")
    operator_dict = dict(zip(operator["OperatorID"].astype(str), operator["OperatorNameZh_tw"]))
    route = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\公路客運路線代碼表.csv")
    route_dict = dict(zip(route["路線唯一識別代碼"], route[["起站中文名稱", "終站中文名稱"]].apply(tuple, axis=1)))
    message_dict = {0 : "未知", 1 : "定期", 2 : "非定期"}
    
    df["Land"] = globe.is_land(df["Latitude"], df["Longitude"]) # Verify if the GPS coordinate is bias(Land or Oscean)
    df = df[df["Land"] == True]
    df["Operator"] = df["OperatorID"].astype(str).map(operator_dict)
    df["車輛方向"] = df["Direction"].map(direction_dict)
    df["車輛狀態"] = df["BusStatus"].map(busStatus_dict)
    df["車輛勤務狀態"] = df["DutyStatus"].map(busDutyStatus_dict)
    df["執勤路線"] = df["RouteUID"].map(route_dict)
    df["班車資訊"] = df["MessageType"].map(message_dict)
    
    df["Coordinates"] = df[["Latitude", "Longitude"]].apply(tuple, axis=1)
    df["NextCoordinates"] = df.groupby("PlateNumb")["Coordinates"].shift(-1)
    df["Distance"] = df.apply(lambda df: geodesic(df["Coordinates"], df["NextCoordinates"]).meters if pd.notna(df["NextCoordinates"]) else 0, axis=1) / 1000
    df = df[df["Distance"] <= 2.5]
    df["Time"] = pd.to_datetime(df["UpdateTime"].map(lambda x: x.split("+")[0]), format="%Y-%m-%dT%H:%M")
    
    result = df.groupby(["PlateNumb", "Operator"]).agg({"Distance": "sum", "Time": lambda x: (x.max() - x.min()) / pd.Timedelta(hours=1)}).reset_index()
    result["Date"] = df["Time"].max().strftime("%Y-%m-%d")
    result["AVG. Speed"] = result["Distance"] / result["Time"]
    result["AVG. Speed"].fillna(0, inplace=True)
    
    return result, df

In [ ]:
df = time_range_data(2023, 1, 5, 18, 0, 2023, 1, 5, 18, 1)
# df.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\20221028國道客運即時測試數據.xlsx", index=False)

In [ ]:
df

In [ ]:
result, raw = icb_summary(df)

In [ ]:
today = datetime.datetime.today().date().strftime("%Y%m%d")

In [ ]:
path = today + "國道客運即時數據.pkl"

In [ ]:
raw.to_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\20221109國道客運即時數據.pkl")
result.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\20221109國道客運即時匯整數據.xlsx", index=False)

In [ ]:
result = icb_summary(df)
# result.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\20221028國道客運即時測試數據_彙整.xlsx", index=False)


## 測試區

In [ ]:
def cb_time_range_data(st_year=0, st_month=0, st_day=0, st_hour=0, st_min=0, end_year=0, end_month=0, end_day=0, end_hour=0, end_min=0):
    cities = ["Taipei", "NewTaipei", "Taoyuan", "Taichung", "Tainan", "Kaohsiung", "Keelung", "Hsinchu", "HsinchuCounty", "MiaoliCounty", "ChanghuaCounty", "NantouCounty", "YunlinCounty", "ChiayiCounty", "ChiayiCounty", "Chiayi", "PingtungCounty", "YilanCounty", "HualienCounty", "TaitungCounty", "KinmenCounty", "PenghuCounty", "LienchiangCounty"]
    city_select = dict()
    for no, city in enumerate(cities):
        print(no, city)
        city_select.setdefault(no, city)
    city = city_select[eval(input("Please type the no. of the city: "))]
    st_time = datetime.datetime(st_year, st_month, st_day, st_hour, st_min).replace(tzinfo=datetime.timezone.utc).astimezone(timezone(timedelta(hours=0))).replace(microsecond=0).isoformat()
    end_time = datetime.datetime(end_year, end_month, end_day, end_hour, end_min).replace(tzinfo=datetime.timezone.utc).astimezone(timezone(timedelta(hours=0))).replace(microsecond=0).isoformat()
    dfs = []
    tdx = TDX()
    while True:
        current_time = datetime.datetime.utcnow().replace(tzinfo=datetime.timezone.utc).astimezone(tz=None).replace(microsecond=0).isoformat()
        if st_time <= current_time <= end_time:
            base_url = "https://tdx.transportdata.tw/api/basic"
            end_point = "/v2/Bus/RealTimeByFrequency/City/{}".format(city) # batch update(Frequency: 60sec.)
            # filter = "Direction eq 1"  # 順逆行: [0:'順行', 1:'逆行']
            url = base_url + end_point
            try:
                result = pd.DataFrame.from_records(tdx.get_response(url))
            except:
                tdx.token = tdx.get_token()
                continue
            else:
                # result["Longitude"] = result["BusPosition"].map(lambda x: x["PositionLon"])
                # result["Latitude"] = result["BusPosition"].map(lambda x: x["PositionLat"])
                dfs.append(result)
                time.sleep(60) # Update the data every 60 sec.
        elif current_time < st_time:
            # print("等待開始")
            continue
        else:
            print("抓取結束")
            result = pd.concat(dfs).reset_index(drop=True)
            # print(result)
            break
    # result = result[["PlateNumb", "OperatorID", "RouteUID", "Direction", "DutyStatus", "BusStatus", "MessageType", "Latitude", "Longitude", "UpdateTime"]]
    return result

In [ ]:
cities = ["Taipei", "NewTaipei", "Taoyuan", "Taichung", "Tainan", "Kaohsiung", "Keelung", "Hsinchu", "HsinchuCounty", "MiaoliCounty", "ChanghuaCounty", "NantouCounty", "YunlinCounty", "ChiayiCounty", "ChiayiCounty", "Chiayi", "PingtungCounty", "YilanCounty", "HualienCounty", "TaitungCounty", "KinmenCounty", "PenghuCounty", "LienchiangCounty"]
base_url = "https://tdx.transportdata.tw/api/basic"
end_point = "/v2/Road/Traffic/Live/VD/City/{}"
tdx = TDX()
# dfs = []
# for city in cities:
#     url = base_url + end_point.format(city)
#     try:
#         df = pd.DataFrame.from_records(tdx.get_response(url))
#     except ValueError:
#         pass
#     except:
#         pass
#     else:
#         dfs.append(df)
# df = pd.concat(dfs).reset_index(drop=True)
# print(df)
url = base_url + end_point.format("Taipei")
df = pd.DataFrame.from_records(tdx.get_response(url))
    
    

In [ ]:
df["VDLives"].apply(pd.Series)["LinkFlows"].apply(pd.Series)

In [ ]:
df["VDLives"].apply(pd.Series)["LinkFlows"].apply(pd.Series)[0].apply(pd.Series)["Lanes"].apply(pd.Series)

In [ ]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
with ThreadPoolExecutor() as executor:
    tpe = executor.submit(cb_time_range_data, 2022, 11, 17, 15, 11, 2022, 11, 17, 15, 52)
    ntp = executor.submit(cb_time_range_data, 2022, 11, 17, 15, 11, 2022, 11, 17, 15, 52)

In [ ]:
test_data = cb_time_range_data(2022, 11, 17, 15, 11, 2022, 11, 17, 15, 40)

In [ ]:
test = test_data.reset_index(drop=True)

In [ ]:
test[(test["OperatorID"] == "100") & (test["PlateNumb"].isin(["KKA-1225", "KKA-1232"]))]

In [ ]:
direction_dict = {0 : "去程(南下)", 1 : "返程(北上)", 2 : "迴圈(雙向)"}
busStatus_dict = {0: "正常", 1 : "車禍", 2 : "故障", 3 : "塞車", 4 : "緊急救援", 5 : "加油", 98 : "偏移路線", 99 : "非營業狀態", 100 : "客滿", 101 : "包車出租", 255 : "未知"}
busDutyStatus_dict = {0: "正常", 1 : "開始", 2 : "結束"}
operator = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\20221026_國道客運業者名單.xlsx")
operator_dict = dict(zip(operator["OperatorID"], operator["OperatorNameZh_tw"]))
route = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\公路客運路線代碼表.csv")
route_dict = dict(zip(route["路線唯一識別代碼"], route[["起站中文名稱", "終站中文名稱"]].apply(tuple, axis=1)))
message_dict = {0 : "未知", 1 : "定期", 2 : "非定期"}
df["Operator"] = df["OperatorID"].map(operator_dict)
df["車輛方向"] = df["Direction"].map(direction_dict)
df["車輛狀態"] = df["BusStatus"].map(busStatus_dict)
df["車輛勤務狀態"] = df["DutyStatus"].map(busDutyStatus_dict)
df["執勤路線"] = df["RouteUID"].map(route_dict)
df["班車資訊"] = df["MessageType"].map(message_dict)